<a href="https://colab.research.google.com/github/hincz-lab/motion-blur-microscopy/blob/main/Training_Material/Split_Into_Tiles/Training_Split_Into_Tiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

This code can be used to split MBM images and their respective masks (colored,labeled, labeled and layered) into tiles that can be used to train the phase one segmentation network.

To begin, let's clone a directory to work in from github.

You will need to go into the next block of code by double clicking the "Show Code" button underneath the "Clone Github Repository" header. Please replace the code <token> with your personal access token. 

To generate a personal access token, in Github, click on your icon at the top right and choose settings, then click on developer settings, and finally, create a personal access token. So, its settings -> Developer Settings -> Personal Access Token (Classic).

After cloning the repository, you can see the repository in the "Files" section of Colab, located on the left-hand side of the screen.

In [ ]:
#@title Clone Github Repository
!git clone https://<token>@github.com/hincz-lab/motion-blur-microscopy.git

fatal: destination path 'motion-blur-microscopy' already exists and is not an empty directory.


Next, we import any packages or libraries that we may need for this code.

In [ ]:
#@title Import Packages And Libraries
import cv2 as cv2
import matplotlib.pyplot as plt
import numpy as np
import os as os

# Complete Analysis

To complete the analysis, you will be required to upload any MBM images or masks that you would like to be split into tiles for training. 

Specifically, if you want to **split MBM images into tiles**, you should upload those images to the directory **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Original_Images**.

If you want to **split MBM full masks**, you should upload those images to the directory **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Full_Masks**

If you want to **split MBM labeled masks**, you should upload those images to the directory **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Labeled_Masks**.

If you want to **split MBM labeled and layered masks**, you should upload those images to the directory **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Labeled_And_Layered_Masks**.

Note, images for the first of these three categories should ideally be uploaded as .png images, but .jpg is acceptable as well. The labeled and layered masks should be uploaded as .npy files.

Further note, that all uploaded files should be titled as integers. If you have MBM images and masks that you need to keep consistent, then all MBM images and masks that correspond to each other should be labeled with the same integer in their respective folders.

In [ ]:
#@title Split Into Tiles

base_Directory = "motion-blur-microscopy/Training_Material/Split_Into_Tiles/"

for subdirectory in os.listdir(base_Directory):
  if '.ipynb_checkpoints' in os.listdir(base_Directory + subdirectory):
    os.rmdir(base_Directory + subdirectory + "/.ipynb_checkpoints")
  if 'blank.txt' in os.listdir(base_Directory + subdirectory):
      os.remove(base_Directory + subdirectory + "/blank.txt")


original_Channel_Path = base_Directory + "Original_Images/"
full_Mask_Path = base_Directory + "Full_Masks/"
labeled_Mask_Path = base_Directory + "Labeled_Masks/"
labeled_And_Layered_Mask_Path = base_Directory + "Labeled_And_Layered_Masks/"

original_Tiles_Path = base_Directory + "Original_Image_Tiles/"
full_Tiles_Path = base_Directory + "Full_Mask_Tiles/"
labeled_Tiles_Path = base_Directory + "Labeled_Tiles/"
labeled_And_Layered_Tiles_Path = base_Directory + "Labeled_And_Layered_Tiles/"


original_Tiles_Resized_Path = base_Directory + "Original_Image_Tiles_Resized/"
full_Tiles_Resized_Path = base_Directory + "Full_Mask_Tiles_Resized/"
labeled_Tiles_Resized_Path = base_Directory + "Labeled_Tiles_Resized/"
labeled_And_Layered_Tiles_Resized_Path = base_Directory + "Labeled_And_Layered_Tiles_Resized/"

tile_Size = 150
tile_Resize_Size = 128


count = 1
for current in os.listdir(original_Channel_Path):
    current_Image = plt.imread(original_Channel_Path + current)
    current = current.replace(".jpg",".png")
    colored_Mask = plt.imread(full_Mask_Path + current)
    labeled_Mask = plt.imread(labeled_Mask_Path + current)
    current = current.replace(".png",".npy")
    labeled_Layered_Mask = np.load(labeled_And_Layered_Mask_Path + current)
    
    channel_Height, channel_Width, channels = np.shape(current_Image)
    vertical_Tiles = int(np.floor(channel_Height/tile_Size))
    horizontal_Tiles = int(np.floor(channel_Width/tile_Size))
    
    for i in range(vertical_Tiles):
        for j in range(horizontal_Tiles):
            original_Image_Tile = current_Image[tile_Size*i:tile_Size + (tile_Size*i),tile_Size*j:tile_Size + (tile_Size*j),:]
            original_Image_Tile_Resized = cv2.resize(original_Image_Tile, (tile_Resize_Size,tile_Resize_Size), interpolation = cv2.INTER_CUBIC)
            plt.imsave(original_Tiles_Path + str(count) + ".png",original_Image_Tile)
            plt.imsave(original_Tiles_Resized_Path + str(count) + ".png",original_Image_Tile_Resized)
            
            colored_Tile = colored_Mask[tile_Size*i:tile_Size + (tile_Size*i),tile_Size*j:tile_Size + (tile_Size*j),:]
            colored_Tile_Resized = cv2.resize(colored_Tile, (tile_Resize_Size,tile_Resize_Size), interpolation = cv2.INTER_NEAREST)
            plt.imsave(full_Tiles_Path + str(count) + ".png",colored_Tile)
            plt.imsave(full_Tiles_Resized_Path + str(count) + ".png",colored_Tile_Resized)
            
            labeled_Tile = labeled_Mask[tile_Size*i:tile_Size + (tile_Size*i),tile_Size*j:tile_Size + (tile_Size*j),:]
            labeled_Tile_Resized = cv2.resize(labeled_Tile, (tile_Resize_Size,tile_Resize_Size), interpolation = cv2.INTER_NEAREST)
            plt.imsave(labeled_Tiles_Path + str(count) + ".png",labeled_Tile)
            plt.imsave(labeled_Tiles_Resized_Path + str(count) + ".png",labeled_Tile_Resized)
            
            labeled_Layered_Tile = labeled_Layered_Mask[tile_Size*i:tile_Size + (tile_Size*i),tile_Size*j:tile_Size + (tile_Size*j)]
            labeled_Layered_Tile_Resized = cv2.resize(labeled_Layered_Tile, (tile_Resize_Size,tile_Resize_Size), interpolation = cv2.INTER_CUBIC)
            np.save(labeled_And_Layered_Tiles_Path + str(count) + ".npy",labeled_Layered_Tile)
            np.save(labeled_And_Layered_Tiles_Resized_Path + str(count) + ".npy",labeled_Layered_Tile_Resized)
            
            count += 1

The output of running the previous block of code will be eight sets of data, 4 sets of data for original, full mask, labeled, and labaeled and layered tiles of size 150x150, and 4 sets of data for original, full mask, labeled, and labeled and layered tiles resized to 128x128.

Tiles of original images can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Original_Image_Tiles**.

Tiles of colored masks can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Full_Mask_Tiles**.

Tiles of labeled masks can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Labeled_Tiles**.

Tiles of labeled and layered masks can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Labeled_And_Layered_Tiles**.

Resized tiles of original images can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Original_Image_Tiles_Resized**.

Resized tiles of colored masks can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Full_Mask_Tiles_Resized**.

Resized tiles of labeled masks can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Labeled_Tiles_Resized**.

Resized tiles of labeled and layered masks can be located at **motion-blur-microscopy -> Training_Material -> Split_Into_Tiles -> Labeled_And_Layered_Tiles_Resized**.